In [ ]:
#need to get gauss1d working collumn 2
#on the big space at the bottom row

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
pi = math.pi

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import cv2
import numpy as np
%matplotlib inline

In [ ]:
fh = open("tiny-imagenet-200/wnids.txt")
count = 0
yval = []
for line in fh:
    nl = ''
    for e in line:
        if e == '\n':
            break
        else:
            nl = nl+e
    yval.append(nl)
    #print(line)
    count+=1
fh.close()



os.chdir('tiny-imagenet-200/train')
#list of folders
t2 = os.listdir()
#yval of list


cwd = os.getcwd()

#gets all x values... need to make y[200] oneshot array
#change t2[:10] for all pics and change x size
class_size = 200
size = 500*class_size


x = []
y = np.zeros([size,class_size])
c = 0
c2 = 0
for d in t2:
    #print(d)
    
    for f2 in os.listdir(d+'/images'):
        x.append(cwd+'\\'+d+'\\images\\'+f2)
        y[c,c2] = 1
        c = c+1
        
    c2+=1   
        
        #print(f2)

In [ ]:
fh = open("tiny-imagenet-200/val/val_annotations.txt")
count = 0
yval = []
yspot = []
for line in fh:
    nl = ''
    nl2 = ''
    for e in line:
        if (e == '\t'):
            count +=1
        elif (count ==0):
            nl2 = nl2 +e
        elif (count == 1): 
            nl = nl+e
        elif count > 1: 
            break
    yval.append(nl)
    yspot.append(nl2)
    #print(line)
    count = 0
fh.close()


os.chdir('tiny-imagenet-200/train')
#list of folders
t3 = os.listdir()

os.chdir('..')
os.chdir('..')

os.chdir('tiny-imagenet-200/val/images')
#list of folders
t2 = os.listdir()
#yval of list


cwd = os.getcwd()

#gets all x values... need to make y[200] oneshot array
#change t2[:10] for all pics and change x size
class_size = 200
size = 10000


x = []
y = np.zeros([size,class_size])
c = 0
c2 = 0
for d in yspot:
    #print(d)
    
    #for f2 in os.listdir(d+'/images'):
    x.append(cwd+'\\'+d)
    y[c, t3.index(yval[c])] = 1
        #y[c,c2] = 1
    c = c+1
        
   # c2+=1   
        
        #print(f2)

In [ ]:
os.chdir('..')
os.chdir('..')
#os.chdir('..')


In [ ]:

print(x[45001])
print(np.nonzero(y[45001]))


In [ ]:
os.getcwd()
ss = 0

In [ ]:
#find where yval is in t2, then set y class as that

x = []
y = np.zeros([size,class_size])
c = 0
c2 = 0
for d in yval:
    #print(d)
    
    for f2 in os.listdir(d+'/images'):
        x.append(cwd+'\\'+d+'\\images\\'+f2)
        y[c,c2] = 1
        c = c+1
        
    c2+=1   
        
        #print(f2)

In [ ]:
import csv
from sklearn.cross_validation import train_test_split
import numpy as np
import argparse
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.models import load_model
import matplotlib.pyplot as plt



from sklearn.utils import shuffle



#X_test = np.sum(X_test/3, axis=3, keepdims=True)
def preprocess(img):
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_out= img.astype('float32')
    #img_out=(img_out-img_out.mean())/img_out.std()
    img_out=img_out/255
    

    return img_out

def generator(x, y, batch_size=128):
    num_samples = len(x)
    x, y = shuffle(x, y)
    while True:
        images = []
        angles = []
        
        for offset in range(0, num_samples, batch_size):
            images = []
            batch_y = y[offset:offset + batch_size]
            batch_x = x[offset:offset + batch_size]
            
            for z in batch_x:
                images.append(cv2.imread(z))
            for h in batch_y:
                angles.append(h)
            X_train = np.array(images)
            y_train = np.array(batch_y)
            X_train = preprocess(X_train)
            
            yield X_train, y_train

train_generator = generator(x,y)

ch, row, col = 3, 160, 320

In [ ]:
b, l=next(train_generator)

In [ ]:

import pandas as pd
from time import time


import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard


In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=54, kernel_size=(3, 3), activation='relu', padding='same',input_shape=(64,64,3)))
model.add(layers.MaxPooling2D())
#model.add(layers.AveragePooling2D(input_shape=(32,32,12)))

model.add(layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))

#model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))

#model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same',activation='relu'))

model.add(layers.MaxPooling2D())

model.add(layers.Flatten())

model.add(layers.Dropout(0.25))

#model.add(layers.Dense(units=4096, activation='relu'))

model.add(layers.Dense(units=4096, activation='relu'))

model.add(layers.Dense(units=2048, activation='relu'))

model.add(layers.Dense(units=class_size, activation = 'softmax'))

#model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
history = model.fit_generator(train_generator, steps_per_epoch=size/128,epochs=15)

os.chdir('..')
os.chdir('..')

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json

with open('model.json', 'r') as f:
    m = model_from_json(f.read())
m.load_weights('model.h5')

In [ ]:
m.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
history = m.evaluate_generator(train_generator, steps = 50)

In [ ]:
out = m.predict(temp2)

In [ ]:
out[0][100]

In [ ]:
temp = cv2.imread("C:\\Users\\jstee\\Desktop\\udacity\\CarND-Traffic-Sign-Classifier-Project\\tiny-imagenet-200\\val\\images\\val_1496.JPEG")
temp3 = cv2.imread("C:\\Users\\jstee\\Desktop\\udacity\\CarND-Traffic-Sign-Classifier-Project\\tiny-imagenet-200\\train\\n03255030\\images\\n03255030_11.JPEG")


In [ ]:
temp3 = temp3.astype('float32')/255
temp = temp.astype('float32')/255

In [ ]:
os.chdir('..')
os.chdir('..')

In [ ]:
plt.imshow(temp)

In [ ]:
plt.imshow(temp3)

In [ ]:
t3[100]

In [ ]:
temp2 = np.expand_dims(temp, axis = 0)

In [ ]:
temp2.shape

In [ ]:
temp2 = np.empty([2,64,64,3])
temp2[0,:,:,:] = temp
temp2[1,:,:,:] = temp3

In [ ]:

temp2.shape

In [ ]:
y[501]